In [1]:
import pandas as pd
import pickle
import time
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
# import matplotlib as plt
from copy import deepcopy
from tqdm.notebook import tqdm as pb


from IPython.core.display import display, HTML
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 100)
display(HTML("<style>.container { width:99% !important; }</style>"))

<ipython-input-1-784d88432367>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Reading in the data

- 1 means positive, 0 negative

## Train Data

In [140]:
df_train = pd.read_json('IE4483_Mini_Project/train.json')

In [141]:
# count number of words
df_train['number_of_words_in_review'] = df_train['reviews'].apply(lambda x: len(x.split(' ')))

In [142]:
df_train['number_of_words_in_review'].max()

1053

In [143]:
# there are more positive reviews than negative reviews in this dataset
# we may adjust this data base on what is in the text itself (area of improvement for model)
df_train['sentiments'].value_counts()

1    6319
0    1082
Name: sentiments, dtype: int64

In [144]:
df_train

,reviews,sentiments,number_of_words_in_review
0,I bought this belt for my daughter in-law for ...,1,14
1,The size was perfect and so was the color. It...,1,20
2,"Fits and feels good, esp. for doing a swim rac...",1,79
3,These socks are absolutely the best. I take pi...,1,56
4,Thank you so much for the speedy delivery they...,1,44
...,...,...,...
7396,"I bought these shirts (black, medium) to wear ...",0,144
7397,"At first, I thought this scarf might not be th...",1,69
7398,I am very picky when it comes to bras. I want...,1,64
7399,"This jacket is wind and water resistant, but n...",0,28


In [145]:
X_train, Y_train = df_train['reviews'].values, df_train['sentiments'].values

## Test Data

In [146]:
df_test = pd.read_json('IE4483_Mini_Project/test.json')

In [147]:
df_test

,reviews
0,I bought 2 sleepers. sleeper had holes in the...
1,I dare say these are just about the sexiest th...
2,"everything about the transaction (price, deliv..."
3,"Not bad for just a shirt. Very durable, and m..."
4,These are truly wrinkle free and longer than t...
...,...
1846,These are stylish and nice and perfect to wear...
1847,We bought these long-sleeved colored shirts fo...
1848,These are really cute and very sexy. They wil...
1849,These shoers were for my daughter. She loves ...


# Data Cleaning (TBC)

In [ ]:
# unbalance dataset, need to find a method to balance it out

In [ ]:
# further formatting of reviews data

# Data Preprocessing

In [148]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

tk = Tokenizer(lower = True)
tk.fit_on_texts(X_train)
X_train_seq = tk.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=300, padding='post')

In [149]:
# split dataset into train, test using existing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_train_pad, Y_train, test_size=0.25, random_state=1)

In [150]:
# split training further into training and validation set
batch_size = 64
X_train1 = X_train[batch_size:]
Y_train1 = Y_train[batch_size:]
X_valid = X_train[:batch_size]
Y_valid = Y_train[:batch_size]

# Creating the Model

In [153]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
vocabulary_size = len(tk.word_counts.keys())+1
max_words = 300
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(400))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [154]:
model.fit(X_train1, Y_train1, validation_data=(X_valid, Y_valid), batch_size=batch_size, epochs=10)

Epoch 1/10
86/86 [==============================] - 4s 35ms/step - loss: 0.4319 - accuracy: 0.8462 - val_loss: 0.3768 - val_accuracy: 0.8750
Epoch 2/10
86/86 [==============================] - 3s 32ms/step - loss: 0.4205 - accuracy: 0.8545 - val_loss: 0.3770 - val_accuracy: 0.8750
Epoch 3/10
86/86 [==============================] - 3s 32ms/step - loss: 0.4168 - accuracy: 0.8545 - val_loss: 0.3788 - val_accuracy: 0.8750
Epoch 4/10
86/86 [==============================] - 3s 31ms/step - loss: 0.4164 - accuracy: 0.8545 - val_loss: 0.3788 - val_accuracy: 0.8750
Epoch 5/10
86/86 [==============================] - 3s 31ms/step - loss: 0.4145 - accuracy: 0.8553 - val_loss: 0.3819 - val_accuracy: 0.8750
Epoch 6/10
86/86 [==============================] - 3s 31ms/step - loss: 0.4224 - accuracy: 0.8531 - val_loss: 0.3770 - val_accuracy: 0.8750
Epoch 7/10
86/86 [==============================] - 3s 31ms/step - loss: 0.4186 - accuracy: 0.8545 - val_loss: 0.3772 - val_accuracy: 0.8750
Epoch 8/10
86

In [155]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Test Accuracy: ", scores[1])

Test Accuracy:  0.8508914113044739


In [156]:
model.predict(X_test, verbose=0)

array([[0.81304675],
       [0.81304663],
       [0.81304675],
       ...,
       [0.81304675],
       [0.81304675],
       [0.81304675]], dtype=float32)

# Use data that did not appear in dataset to check model

In [133]:
df_test.shape

(1851, 1)

In [135]:
X_check = df_test['reviews'].values
X_check_seq = tk.texts_to_sequences(X_check)
X_check_pad = pad_sequences(X_check_seq, maxlen=100, padding='post')

X_predict = model.predict(X_check_pad, verbose=0)

In [136]:
df_test['sentiment'] = X_predict

In [137]:
df_test['sentiment'] = df_test['sentiment'].apply(lambda x: round(x))

In [138]:
df_test['sentiment'].value_counts()

1    1844
0       7
Name: sentiment, dtype: int64

In [65]:
df_test.to_csv('results.csv', index=False)